In [25]:
import os
from parsons import VAN, Redshift, Table
from datetime import datetime, timedelta
from canalespy import logger, setup_environment

In [26]:
setup_environment()
rs = Redshift()
van = VAN(api_key='f49f4c62-439c-362d-eae2-e6b32ccb8431', db='EveryAction')

In [11]:
van.get_event(event_id=750017882, expand_fields=['locations', 'codes', 'shifts', 'roles', 'notes', 'financialProgram', 'ticketCategories', 'voterRegistrationBatches'])

events INFO Found event 750017882.


{'eventId': 750017882,
 'name': 'New Mode Direct Action',
 'shortName': 'New Mode DA',
 'description': None,
 'startDate': '2021-09-15T20:30:00-04:00',
 'endDate': '2021-09-15T21:00:00-04:00',
 'eventType': {'eventTypeId': 164850, 'name': 'Action-DirectAction'},
 'isOnlyEditableByCreatingUser': False,
 'isPubliclyViewable': None,
 'locations': [{'locationId': 107,
   'name': 'Virtual',
   'displayName': 'Virtual',
   'address': None,
   'id': 107,
   'notes': None,
   'codes': None}],
 'codes': [],
 'notes': [],
 'shifts': [{'eventShiftId': 34195,
   'name': 'Single Shift',
   'startTime': '2021-09-15T20:30:00-04:00',
   'endTime': '2021-09-15T21:00:00-04:00'}],
 'roles': [{'roleId': 100525,
   'name': 'Action leader',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100526,
   'name': 'Attendee',
   'isEventLead': False,
   'min': None,
   'max': None,
   'goal': None},
  {'roleId': 100527,
   'name': 'Chant Leader',
   'isEventLead': False,
   '

In [32]:
getevent = rs.query(f'''
    select * from txop_newmode.ea_event_summary
    where dateoffsetbegin::date = '{str(test_v4['action_date_date'][0])}'
''')

In [33]:
getevent

eventid,eventcalendarid,eventcalendarname,eventname,dateoffsetbegin,dateoffsetend,eventshiftid
750015272,128562,Member Doing Work,Member Doing Work,2021-04-14,2021-04-14,31579
750017860,164850,Action-DirectAction,New Mode Direct Action,2021-04-14,2021-04-14,34173


In [37]:
getevent['eventid']

[750015272, 750017860]

In [24]:
str(test_v4['action_date_date'][0])

'2021-04-14'

In [27]:
TIMESTAMP = datetime.now() #timestamp to be used for logtable

In [64]:
LOG_TABLE = 'txop_newmode.calls_to_ea_log'
SOURCE_TABLE = 'txop_newmode.call_summary_to_ea'

In [65]:
if not rs.table_exists(LOG_TABLE): #if Log Table does not already exist, create it
    # standard sql
    sql = "(outreach_id int, vanid int, date_created timestamp, sr_load varchar(100), event_load varchar(100));"
    rs.query(f"create table {LOG_TABLE} {sql}")
    logger.info(f"Creating {LOG_TABLE}...")

In [85]:
source_tbl = rs.query(f'''
  SELECT a.*
      , e.eventid
      , e.eventshiftid
  FROM (select *, row_number() over(partition by outreach_id order by action_date desc) as dup
    from {SOURCE_TABLE}) a
  LEFT JOIN {EA_EVENT_SUMMARY} e on e.dateoffsetbegin::date = a.action_date::date and e.eventcalendarid = a.event_calendar_id
  LEFT JOIN {LOG_TABLE} l using(outreach_id)
  where
  l.outreach_id is null
  and dup=1
  ;
  '''
)

In [86]:
logger.info(f"{source_tbl.num_rows} rows found.")

INFO 16 rows found.
INFO:canalespy:16 rows found.


In [87]:
if source_tbl.num_rows > 0:
    logger.info(f"{source_tbl.num_rows} new records found. Inserting into table")
    loaded = [['outreach_id', 'vanid', 'datecreated','sr_load', 'signup_id']]

INFO 16 new records found. Inserting into table
INFO:canalespy:16 new records found. Inserting into table


In [88]:
    # 1. Convert action date to iso format, and add it to a new column
    source_tbl.add_column(
      'action_date_iso',
      lambda row: datetime.strptime(row['action_date'], '%Y-%m-%d %H:%M:%S').isoformat()
    )

    # 2. Convert action date to datetime date, and add it to a new column
    source_tbl.add_column(
      'action_date_date',
      lambda row: datetime.strptime(row['action_date'], '%Y-%m-%d %H:%M:%S').date()
    )

    # 3. Substract one day from the action date, convert the output to a string, and add it to a new column
    source_tbl.add_column(
      'action_date_after',
      lambda row: str(row['action_date_date'] - timedelta(days=1))
    )

    # 4. Add/substract one day from the action date, convert the output to a string, and add it to a new column
    source_tbl.add_column(
      'action_date_before',
      lambda row: str(row['action_date_date'] + timedelta(days=1))
    )

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,event_calendar_id,call_status,call_hangup_reason,day_rank,dup,eventid,eventshiftid,action_date_iso,action_date_date,action_date_after,action_date_before
10464744,2021-04-14 14:01:38,Denise,Hernandez,hernandez.denise210@gmail.com,2102877852,78213,https://organizetexas.org/right2j/hb829-will-enforce-police-accountability?fbclid=IwAR1l8XO3SsPMEJvV6PBdz-C8QdVPxGHOQwv2n0XXBRN6S_S-4pveJaJjbyc,914328,MDW,128562,completed,Call ended successfully,1,1,750015272,31579,2021-04-14T14:01:38,2021-04-14,2021-04-13,2021-04-15
10488863,2021-04-15 21:33:20,Patti,Peacock,pattip49@hotmail.com,5127910773,78613,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=72dbda8f5e292c965b8ab946e37f6306&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,1,750015273,31580,2021-04-15T21:33:20,2021-04-15,2021-04-14,2021-04-16
10486574,2021-04-15 17:59:17,Elizabeth,Gregory,elizabeth_gregory@earthlink.net,7132568310,77030,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=2c8e13b63ecd188ddb63b439abaa755c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,1,750015273,31580,2021-04-15T17:59:17,2021-04-15,2021-04-14,2021-04-16
10486560,2021-04-15 17:57:59,Anita,Garcia,gar04317@yahoo.com,5128261371,78749,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=907fa04fcace94d5702469b052cd834c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,1,750015273,31580,2021-04-15T17:57:59,2021-04-15,2021-04-14,2021-04-16
10486721,2021-04-15 18:10:24,barbara,baruch,barbarabaruch@gmail.com,2812160858,78255,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=7b19d91cfdd42508f61ea3ea3bf5ceb8&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,1,750015273,31580,2021-04-15T18:10:24,2021-04-15,2021-04-14,2021-04-16


NameError: name 'NULL' is not defined

[]

In [70]:
EA_EVENT_SUMMARY = 'txop_newmode.ea_event_summary'
SURVEY_QUESTION_ID = 163740 # issue campaign
CONTACT_TYPE_ID = 1 # phone
#### UPDATE IN CODE
EVENT_TYPE_ID_MDW = 128562 # MDW
EVENT_TYPE_ID_DA = 164850 # DA
ROLE_ID = 100541 # talked to politician
STATUS_ID = 2 # complete

In [38]:
for ppl in source_tbl:
    logger.info(f"Working on outreach_id={ppl['outreach_id']}")
    
    if ppl['day_rank'] == 1:

        find_event = rs.query(f'''
            SELECT 
                eventid
              , eventshiftid 
            FROM {EA_EVENT_SUMMARY}
            WHERE dateoffsetbegin::date = '{str(ppl['action_date_date'])}'
            AND eventcalendarid = '{EVENT_TYPE_ID_MDW}'
        ''')
        print(find_event)
        
        event_id = find_event['eventid'][0]
        print(event_id)

INFO Working on outreach_id=10496281
INFO:canalespy:Working on outreach_id=10496281
INFO Working on outreach_id=10487814
INFO:canalespy:Working on outreach_id=10487814
INFO Working on outreach_id=10487436
INFO:canalespy:Working on outreach_id=10487436


{'eventid': 750015273, 'eventshiftid': 31580}
750015273


INFO Working on outreach_id=10464744
INFO:canalespy:Working on outreach_id=10464744


{'eventid': 750015273, 'eventshiftid': 31580}
750015273


INFO Working on outreach_id=10488863
INFO:canalespy:Working on outreach_id=10488863


...

{'eventid': 750015273, 'eventshiftid': 31580}
750015273


INFO Working on outreach_id=10488886
INFO:canalespy:Working on outreach_id=10488886


{'eventid': 750015273, 'eventshiftid': 31580}
750015273


INFO Working on outreach_id=10486866
INFO:canalespy:Working on outreach_id=10486866


{'eventid': 750015273, 'eventshiftid': 31580}
750015273
{'eventid': 750015273, 'eventshiftid': 31580}
750015273


In [44]:
ea_event_summary = rs.query(f"select * from {EA_EVENT_SUMMARY}")

In [58]:
ea_event_summary

eventid,eventcalendarid,eventcalendarname,eventname,dateoffsetbegin,dateoffsetend,eventshiftid
750017859,164850,Action-DirectAction,New Mode Direct Action,2021-04-07,2021-04-07,34172
750015225,128562,Member Doing Work,Member Doing Work,2021-02-26,2021-02-26,31532
750015173,128562,Member Doing Work,Member Doing Work,2021-01-05,2021-01-05,31480
750015185,128562,Member Doing Work,Member Doing Work,2021-01-17,2021-01-17,31492
750015184,128562,Member Doing Work,Member Doing Work,2021-01-16,2021-01-16,31491


In [78]:
event_test = rs.query(f'''
            SELECT s.*
              , e.eventid
              , e.eventshiftid 
            FROM {SOURCE_TABLE} s
            LEFT JOIN {EA_EVENT_SUMMARY} e on e.dateoffsetbegin::date = s.action_date::date and e.eventcalendarid = s.event_calendar_id
        ''')

In [80]:
event_test

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,event_calendar_id,call_status,call_hangup_reason,day_rank,eventid,eventshiftid
10215647,2021-03-23 17:38:25,Justin,Leaca,justin.lecea@gmail.com,8108143304,78229,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,128562,completed,Call ended successfully,1,750015250,31557
10488886,2021-04-15 21:36:59,ROBERT,KRAKOW,rkrakow@gibsondunn.com,2145057124,75209,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=e66820eaae2b16cc833dd1b355930af2&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,750015273,31580
10486560,2021-04-15 17:57:59,Anita,Garcia,gar04317@yahoo.com,5128261371,78749,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=907fa04fcace94d5702469b052cd834c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,750015273,31580
10486574,2021-04-15 17:59:17,Elizabeth,Gregory,elizabeth_gregory@earthlink.net,7132568310,77030,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=2c8e13b63ecd188ddb63b439abaa755c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,750015273,31580
10487363,2021-04-15 19:07:51,Melba,Vasquez,melvasquez@aol.com,5123274218,78746,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=f4d1a233b0d88b20dc4a486a7a68a670&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,128562,completed,Call ended successfully,1,750015273,31580


In [60]:
#     eventid
#   , eventshiftid 
# FROM {EA_EVENT_SUMMARY}
# WHERE dateoffsetbegin::date = '{str(ppl['action_date_date'])}'
# AND eventcalendarid = '{EVENT_TYPE_ID_MDW}'

# source_tbl.add_column(
#   'event_id',
#   ea_event_summary.select_rows(lambda row: str(row[dateoffsetbegin] == source_tbl['action_date_date']))
# )


# find_dates = source_tbl.add_column(
#     'event_id',
#     ea_event_summary['eventid'].map(lambda e: e['dateoffsetbegin'] == source_tbl['action_date_date'])
# )

find_dates['event_id'] = ea_event_summary['eventid'].select_rows(lambda e: e['dateoffsetbegin'] == source_tbl['action_date_date'])

AttributeError: 'list' object has no attribute 'select_rows'

In [50]:
find_dates

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank,dup,vanid,date_created,sr_load,signup_id,action_date_iso,action_date_date,action_date_after,action_date_before,event_id
10496281,2021-04-16 14:43:18,Jean,Leider,jeancaffeine@gmail.com,5129234959,78745,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=7aad08a3c87f20f6eb9401c694c31c19&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,3,1,None,None,None,None,2021-04-16T14:43:18,2021-04-16,2021-04-15,2021-04-17,"{'eventid': 750017859, 'eventcalendarid': 164850, 'eventcalendarname': 'Action-DirectAction', 'eventname': 'New Mode Direct Action', 'dateoffsetbegin': datetime.date(2021, 4, 7), 'dateoffsetend': datetime.date(2021, 4, 7), 'eventshiftid': 34172} {'eventid': 750015225, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 2, 26), 'dateoffsetend': datetime.date(2021, 2, 26), 'eventshiftid': 31532} {'eventid': 750015173, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 5), 'dateoffsetend': datetime.date(2021, 1, 5), 'eventshiftid': 31480} {'eventid': 750015185, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 17), 'dateoffsetend': datetime.date(2021, 1, 17), 'eventshiftid': 31492} {'eventid': 750015184, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 16), 'dateoffsetend': datetime.date(2021, 1, 16), 'eventshiftid': 31491} ..."
10487814,2021-04-15 19:51:07,Maria,Smith,claudiasmith469@gmail.com,4696003616,75212,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=2&can_id=02cd61931b9ec6dc6d8672c37e1c12f2&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1,1,None,None,None,None,2021-04-15T19:51:07,2021-04-15,2021-04-14,2021-04-16,"{'eventid': 750017859, 'eventcalendarid': 164850, 'eventcalendarname': 'Action-DirectAction', 'eventname': 'New Mode Direct Action', 'dateoffsetbegin': datetime.date(2021, 4, 7), 'dateoffsetend': datetime.date(2021, 4, 7), 'eventshiftid': 34172} {'eventid': 750015225, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 2, 26), 'dateoffsetend': datetime.date(2021, 2, 26), 'eventshiftid': 31532} {'eventid': 750015173, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 5), 'dateoffsetend': datetime.date(2021, 1, 5), 'eventshiftid': 31480} {'eventid': 750015185, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 17), 'dateoffsetend': datetime.date(2021, 1, 17), 'eventshiftid': 31492} {'eventid': 750015184, 'eventcalendarid': 128562, 'eventcalendarname': 'Member Doing Work', 'eventname': 'Member Doing Work', 'dateoffsetbegin': datetime.date(2021, 1, 16), 'dateoffsetend': datetime.date(2021, 1, 16), 'eventshiftid': 31491} ..."
10487436,2021-04-15 19:13:20,ida,miller,idafrae@gmail.com,5127385998,78666,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=3a4ddd37f7e1617ba79f2a899ddcbd02&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_

In [12]:
test = rs.query(f"select * from {SOURCE_TABLE}")

In [13]:
test

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank
10464744,2021-04-14 14:01:38,Denise,Hernandez,hernandez.denise210@gmail.com,2102877852,78213,https://organizetexas.org/right2j/hb829-will-enforce-police-accountability?fbclid=IwAR1l8XO3SsPMEJvV6PBdz-C8QdVPxGHOQwv2n0XXBRN6S_S-4pveJaJjbyc,914328,MDW,completed,Call ended successfully,1
10488886,2021-04-15 21:36:59,ROBERT,KRAKOW,rkrakow@gibsondunn.com,2145057124,75209,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=e66820eaae2b16cc833dd1b355930af2&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10486560,2021-04-15 17:57:59,Anita,Garcia,gar04317@yahoo.com,5128261371,78749,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=907fa04fcace94d5702469b052cd834c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10486574,2021-04-15 17:59:17,Elizabeth,Gregory,elizabeth_gregory@earthlink.net,7132568310,77030,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=2c8e13b63ecd188ddb63b439abaa755c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10488023,2021-04-15 20:10:33,Holly,herzer,hlherz@aol.com,2147241670,78613,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=7f9a1a6dfca67e4fdb37a8b316b2eddf&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1


In [122]:
test_v3 = test.add_column(
    'action_date_date',
    lambda row: row['action_date'].datetime.strptime('%Y-%m-%d %H:%M:%S').date()
)

ValueError: Column action_date_date already exists

In [14]:
type(test['action_date'][0])

str

In [15]:
test_v4 = test

In [16]:
test_v4

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank
10464744,2021-04-14 14:01:38,Denise,Hernandez,hernandez.denise210@gmail.com,2102877852,78213,https://organizetexas.org/right2j/hb829-will-enforce-police-accountability?fbclid=IwAR1l8XO3SsPMEJvV6PBdz-C8QdVPxGHOQwv2n0XXBRN6S_S-4pveJaJjbyc,914328,MDW,completed,Call ended successfully,1
10488886,2021-04-15 21:36:59,ROBERT,KRAKOW,rkrakow@gibsondunn.com,2145057124,75209,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=e66820eaae2b16cc833dd1b355930af2&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10486560,2021-04-15 17:57:59,Anita,Garcia,gar04317@yahoo.com,5128261371,78749,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=907fa04fcace94d5702469b052cd834c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10486574,2021-04-15 17:59:17,Elizabeth,Gregory,elizabeth_gregory@earthlink.net,7132568310,77030,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=2c8e13b63ecd188ddb63b439abaa755c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1
10488023,2021-04-15 20:10:33,Holly,herzer,hlherz@aol.com,2147241670,78613,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=7f9a1a6dfca67e4fdb37a8b316b2eddf&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1


In [17]:
type(test_v4['action_date'])

list

In [18]:
type(test_v4['action_date'][0])

str

In [174]:
test_v4.add_column(
    'action_date_iso',
    lambda row: datetime.strptime(row['action_date'], '%Y-%m-%d %H:%M:%S').isoformat()
)

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank,action_date_after_3,action_date_iso
10215647,2021-03-23 17:38:25,Justin,Leaca,justin.lecea@gmail.com,8108143304,78229,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,Call ended successfully,1,2021-03-23T17:38:25,2021-03-23T17:38:25


In [19]:
test_v4.add_column(
    'action_date_date',
    lambda row: datetime.strptime(row['action_date'], '%Y-%m-%d %H:%M:%S').date()
)

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank,action_date_date
10464744,2021-04-14 14:01:38,Denise,Hernandez,hernandez.denise210@gmail.com,2102877852,78213,https://organizetexas.org/right2j/hb829-will-enforce-police-accountability?fbclid=IwAR1l8XO3SsPMEJvV6PBdz-C8QdVPxGHOQwv2n0XXBRN6S_S-4pveJaJjbyc,914328,MDW,completed,Call ended successfully,1,2021-04-14
10488886,2021-04-15 21:36:59,ROBERT,KRAKOW,rkrakow@gibsondunn.com,2145057124,75209,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=e66820eaae2b16cc833dd1b355930af2&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1,2021-04-15
10486560,2021-04-15 17:57:59,Anita,Garcia,gar04317@yahoo.com,5128261371,78749,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=907fa04fcace94d5702469b052cd834c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1,2021-04-15
10486574,2021-04-15 17:59:17,Elizabeth,Gregory,elizabeth_gregory@earthlink.net,7132568310,77030,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=2c8e13b63ecd188ddb63b439abaa755c&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1,2021-04-15
10488023,2021-04-15 20:10:33,Holly,herzer,hlherz@aol.com,2147241670,78613,https://organizetexas.org/healthcare/call-speaker-phelan-now?preview=true&link_id=3&can_id=7f9a1a6dfca67e4fdb37a8b316b2eddf&source=email-action-alert-new-momentum-on-a-healthcare-expansion-bill-at-the-legislature-2&email_referrer=email_1142928&email_subject=action-alert-call-speaker-phelan-now,695590,MDW,completed,Call ended successfully,1,2021-04-15


In [36]:
test_v4['action_date_date'][0]

datetime.date(2021, 4, 14)

In [177]:
test_v4.add_column(
    'action_date_after',
    lambda row: str(row['action_date_date'] - timedelta(days=1))
)

outreach_id,action_date,first_name,last_name,email,phone,zip,parent_url,survey_response_id,event_type,call_status,call_hangup_reason,day_rank,action_date_after_3,action_date_iso,action_date_date,action_date_after
10215647,2021-03-23 17:38:25,Justin,Leaca,justin.lecea@gmail.com,8108143304,78229,https://organizetexas.org/r2j/letvotersdecide,914328,MDW,completed,Call ended successfully,1,2021-03-23T17:38:25,2021-03-23T17:38:25,2021-03-23,2021-03-22


In [147]:
test_v4

RecursionError: maximum recursion depth exceeded

RecursionError: maximum recursion depth exceeded while calling a Python object

In [46]:
if source_tbl.num_rows > 0:
    logger.info(f"{source_tbl.num_rows} new records found. Inserting into table")
    loaded = [['outreach_id', 'vanid', 'datecreated','sr_load', 'event_load']] #column names for our eventual Parsons Table of what we've loaded

    for ppl in source_tbl:
      logger.info(f"Working on outreach_id={ppl['outreach_id']}")
      
      # find contacts in ea
      ea = van.find_person(first_name=ppl['first_name']
                          ,last_name=ppl['last_name']
                          ,email=ppl['email']
                          ,phone=ppl['phone']
                          ,zip=ppl['zip'])

      if isinstance(ea, tuple):
        logger.info("Issue loading to VAN!")
        vanids = [ppl['vanid'], 0, TIMESTAMP, 'Error', 'Error', 'Error', 'Error']
        continue

      logger.info(f"Found in EveryAction as {ea['vanId']}...")

      # Survey Responses
      try:
          van.apply_survey_response(id=ea['vanId'],
                                    id_type='vanid',
                                    survey_question_id='163740',
                                    survey_response_id=ppl['survey_response_id'],
                                    contact_type_id='133', # relational call, phone = 1
                                    date_canvassed=ppl['action_date'])
      except:
          logger.info(f"Couldn't apply survey response")

      # Events (MDW only)
      signup_id = None
      if ppl['day_rank'] == 1:
        # convert action date to a datetime date object
        action_date = datetime.strptime(ppl['action_date'], '%Y-%m-%d %H:%M:%S').date()
        # create variables to add and substract one day from the action date and convert the output to a string
        action_date_after = str(action_date - timedelta(days=1))
        action_date_before = str(action_date + timedelta(days=1))

        # find the recurring mdw event that occurs on the day the new/mode action took place and store the event_id as a variable
        find_mdw_event = van.get_events(event_type_ids=128562,starting_after={action_date_after}, starting_before={action_date_before})              
        event_id = find_mdw_event['eventId']

        # find the shift id for our event
        shifts = find_mdw_event['shifts'][0][0]
        # store shift id
        shift_id = shifts['eventShiftId']

        # static event values
        role_id = 100541 
        status_id = 2 # status = completed

        try: 
          signup_id = van.create_signup(vanid=ea['vanId'],
                            event_id=event_id, 
                            shift_id = shift_id, 
                            role_id = role_id, 
                            status_id = status_id)
        except: 
          logger.info(f"Couldn't create event signup")

      # Create list of key information for log table
      vanids = [ppl['outreach_id'], ea['vanId'],TIMESTAMP,ppl['survey_response_id'],signup_id]

      # Append list to log table
      loaded.append(vanids)

    logger.info(f"Finished the loop! Now to load the work we've done into {LOG_TABLE}")
    Table(loaded).to_redshift(LOG_TABLE,if_exists='append')
else:
  # if there were 0 records from initial query
  logger.info("No new data to send to EveryAction!")

INFO 1 new records found. Inserting into table
INFO Working on outreach_id=10215647
people INFO Finding Justin  Leaca .
INFO Found in EveryAction as 102021797...
survey_questions INFO Applying survey question 163740 to vanid 102021797
INFO Couldn't apply survey response
events INFO Found 1 events.
INFO Couldn't create event signup
INFO Finished the loop! Now to load the work we've done into txop_newmode.calls_to_ea_log


ParamValidationError: Parameter validation failed:
Invalid bucket name "parsons-tmc export": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$"